# SPECTRE-CPU-V2

# SETUP PRE-REQUISITES

In [1]:
import os
import platform
import sys
import glob

import tensorflow as tf
#from tensorflow.keras import layers

#import keras
from tensorflow import keras
from keras import layers
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, Flatten, Input
from keras.models import Model
from keras import layers, models

from sklearn.model_selection import train_test_split

import pandas as pd

from tabulate import tabulate

def escape():
    sys.exit()

2023-05-03 03:58:44.725717: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 03:58:44.752133: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 03:58:44.752697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 03:58:45.416065: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")

Python Platform: Linux-6.2.13-zen-1-zen-x86_64-with-glibc2.37
Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.9.16 (main, Mar  8 2023, 14:00:05) 
[GCC 11.2.0]


Setup INFO level

In [3]:
tf.get_logger().setLevel('INFO')

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

# ANN TRAINING

## Dataset Import

Change: Dataset format used is CSV

Test CSV Dataset Headers

In [4]:
# Test Header of the CSV if there is error, run the CICFlowMeter again

def read_header(file_path):
    return pd.read_csv(file_path, nrows=0).columns

directory = "/home/aryn/spectre-dev/dataset/ISCX-IDS-2012/PCAP-CSV/"

# Get all CSV files in the directory
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Read headers from all CSV files
headers = [read_header(os.path.join(directory, f)) for f in csv_files]

# Check if all headers match
if all(h.equals(headers[0]) for h in headers[1:]):
    print("Headers match")
else:
    print("Headers do not match")

Headers match


Create the dataset with `tf.data.experimental.make_csv_dataset`

In [5]:
# Get the list of CSV files
csv_files = glob.glob("/home/aryn/spectre-dev/dataset/ISCX-IDS-2012/PCAP-CSV/*.csv")

In [6]:
# Read the header from one of the CSV files to get column names
with open(csv_files[0], "r") as f:
    header_line = f.readline().strip()
    column_names = header_line.split(',')
    
print(column_names)

print(len(column_names))

['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',

Check the dataset length

In [7]:
file_pattern = "/home/aryn/spectre-dev/dataset/ISCX-IDS-2012/PCAP-CSV/*.csv"
csv_files = glob.glob(file_pattern)

dataset_size = 0

for csv_file in csv_files:
    with open(csv_file, "r") as f:
        # Subtract 1 to account for the header row
        num_records = sum(1 for _ in f) - 1
        dataset_size += num_records

print("Total records in the dataset:", dataset_size)

Total records in the dataset: 3612546


In [8]:
data_array = tf.data.experimental.make_csv_dataset(
    file_pattern=csv_files,  # Use the list of CSV file paths
    batch_size=2,
    label_name="Flow ID",
    header=True,
    num_epochs=1,
    ignore_errors=True,
    field_delim=',',
    shuffle=True,
    column_names=column_names  # Use the column_names list
)

Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


2023-05-03 03:59:38.934513: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-05-03 03:59:38.934551: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: aryn-eos
2023-05-03 03:59:38.934558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: aryn-eos
2023-05-03 03:59:38.934780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 530.41.3
2023-05-03 03:59:38.934808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.41.3
2023-05-03 03:59:38.934817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 530.41.3


Dataset Output 

In [9]:
# Dataset Output

# Unbatch the dataset
unbatched_data = data_array.unbatch()

# Take the first 2 elements
dataset_xray = unbatched_data.take(1)

# Print the first 2 elements with proper formatting
for features, label in dataset_xray:
    print("Flow ID: {}".format(label.numpy()))
    print("Features:")
    for key, value in features.items():
        print("  {!r:20s}: {}".format(key, value.numpy()))
    print("\n")


2023-05-03 03:59:43.409588: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype string and shape [1]
	 [[{{node Placeholder/_8}}]]
2023-05-03 03:59:43.410959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_52' with dtype float and shape [1]
	 [[{{node Placeholder/_52}}]]


Flow ID: b'8.6.0.1-8.0.6.4-0-0-0'
Features:
  'Src IP'            : b'8.6.0.1'
  'Src Port'          : 0
  'Dst IP'            : b'8.0.6.4'
  'Dst Port'          : 0
  'Protocol'          : 0
  'Timestamp'         : b'13/06/2010 07:40:55 PM'
  'Flow Duration'     : 40004026
  'Total Fwd Packet'  : 2
  'Total Bwd packets' : 0
  'Total Length of Fwd Packet': 0.0
  'Total Length of Bwd Packet': 0.0
  'Fwd Packet Length Max': 0.0
  'Fwd Packet Length Min': 0.0
  'Fwd Packet Length Mean': 0.0
  'Fwd Packet Length Std': 0.0
  'Bwd Packet Length Max': 0.0
  'Bwd Packet Length Min': 0.0
  'Bwd Packet Length Mean': 0.0
  'Bwd Packet Length Std': 0.0
  'Flow Bytes/s'      : 0.0
  'Flow Packets/s'    : 0.04999496787786484
  'Flow IAT Mean'     : 40004024.0
  'Flow IAT Std'      : 0.0
  'Flow IAT Max'      : 40004024.0
  'Flow IAT Min'      : 40004024.0
  'Fwd IAT Total'     : 40004024.0
  'Fwd IAT Mean'      : 40004024.0
  'Fwd IAT Std'       : 0.0
  'Fwd IAT Max'       : 40004024.0
  'Fwd IAT Mi

## Training

In [10]:
dataset_size = dataset_size
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train_data = data_array.take(train_size)
test_data = data_array.skip(train_size)

In [11]:
# Inspect the first element to get the input shape
for features, labels in train_data.take(1):
    input_shape = list(features.values())[0].shape

num_classes = 84

# Create the ANN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=input_shape),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_classes, activation='softmax')
])


2023-05-03 04:00:00.557183: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_44' with dtype int32 and shape [1]
	 [[{{node Placeholder/_44}}]]
2023-05-03 04:00:00.558398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_25' with dtype float and shape [1]
	 [[{{node Placeholder/_25}}]]


In [17]:
# Model Summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                192       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 84)                2772      
                                                                 
Total params: 5,044
Trainable params: 5,044
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
def preprocess_data(features, labels):
    features_values = [tf.cast(value, tf.float32) for key, value in features.items() if key != 'Label']
    return tf.stack(features_values, axis=-1), labels


In [13]:
train_data = train_data.map(preprocess_data)
test_data = test_data.map(preprocess_data)

In [14]:
train_data_batched = train_data.batch(32)
test_data_batched = test_data.batch(32)

In [15]:
# Batch the dataset
#train_data_batched = data_array.batch(32)

In [16]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Train the model
model.fit(train_data_batched, epochs=10)

2023-05-03 04:01:40.085745: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_56' with dtype int32 and shape [1]
	 [[{{node Placeholder/_56}}]]
2023-05-03 04:01:40.086974: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_37' with dtype float and shape [1]
	 [[{{node Placeholder/_37}}]]
2023-05-03 04:01:40.150762: W tensorflow/core/framework/op_kernel.cc:1807] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: {{function_node __wrapped__MakeIterator_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cast string to float is not supported
	 [[{{node Cast}}]] [Op:MakeIterator]

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_data.batch(32))

# EXPORT MODEL

In [ ]:
# Export as SavedModel
tf.saved_model.save(my_model, '/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/SavedModel/')

# Export as Keras Model
my_model.save("/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/spectre_ddos_hd5")

# Export as Keras H5 Model
my_model.save("/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/spectre_ddos_h5.h5")

# ANN ACTIVATION GRAPH